# Studying Human Cognition Using Data Science

This notebook contains the demo code required for the lecture "Studying Human Cognition Using Data Science".

A demo of the main empirical task used to generate the data is available at https://bit.ly/gg-problem-solving-demo.

We will use three python libraries:
- [Pandas](https://pandas.pydata.org), a popular tool for data manipulation and basic analyses
- [Numpy](https://numpy.org/), a widely-used library for numerical computing in python
- [Matplotlib](https://matplotlib.org/), the de-facto standard tool for data visualisation in python
- [Scikit-learn](https://scikit-learn.org/stable/index.html), a collection of useful statistical models and other tools for data analysis

Make sure your python environment (or virtual environment) has all three libraries installed before continuing!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1 - Data Loading and Inspection

We start by loading the data from the local filesystem.

The data is stored in "CSV" format, a simple (but inefficient) data format based on plain text. "CSV" means "comma-separated values". If you open the file in a text editor, you'll see many strings of text each separated by a comma. Each line is like a row of a table, and the commas separate different columns.

To process the data, we will first need to load it into python and convert all the strings of text to the right data types (e.g. float, boolean, string). Fortunately, Pandas gives us an easy way to do so: `pd.read_csv`.

In [ ]:
study_df = pd.read_csv("./demo-project-study-data.csv")
task_features_df = pd.read_csv("./demo-project-task-features.csv")

### Study Data

Let's inspect the content of the `study_df` dataframe to make sure the content has loaded in the right format. Pandas dataframes have a `.dtypes` property that we can use for this purpose.

In [ ]:
study_df.dtypes

We can see that there are 16 columns in the dataframe. Pandas has inferred suitable types for each one: integer types (for whole numbers), float types (for decimal numbers), and object types (for strings).

Now let's look at the first few rows of the dataframe, to see how it's structured:

In [ ]:
study_df.head(5)

Most of the columns contain data that we won't need. The most important columns for our research question are:
- `participant_id`: The unique identifier assigned to each participant, persistently across sessions of the study.
- `trial_number`: The order in which this trial (puzzle task) occurred in the session.
- `event_timestamp`: The time in milliseconds at which the recorded event occurred, relative to the start of the trial.
- `event_type`: What kind of event is being recorded: trial start/end, selection or deselection of a rule or symbol, application of a rule, successful completion of a trial.
- `task_signature`: A tuple containing the start string and target string of the current trial. Each puzzle is uniquely specified by this tuple of start and target states.

We can see all the different possible values for a column of the dataframe by using the `unique()` method:

In [ ]:
study_df.event_type.unique()

Let's see how many of the trials were completed successfully.
All trials end with a "trial:end" event, and the successful ones also have a "goal:achieved" event.

Complete the code below to compute the proportion of trials that were solved.

In [ ]:
# How many trials were there in total
n_trials = len(study_df[study_df.event_type == "trial:end"])

# How many trials were successful
n_successes = len(study_df[study_df.event_type == "goal:achieved"])

# The proportion of all trials that were completed successfully
proportion_trials_successful = ?

print(f"Out of {n_trials} trials, {n_successes} ({proportion_trials_successful:.2%}) were completed successfully")

### Task Features

Now let's look at the other dataframe.

In [ ]:
print(f"The task features dataframe has {len(task_features_df.columns)} columns, and {len(task_features_df)} rows")

In [ ]:
task_features_df.head(5)

This dataframe contains pre-computed features for each puzzle task. There are 72 puzzles, and for each one I have calculated 20 features describing their properties (e.g. how long is the start state; how many steps are needed to solve the puzzle, etc).

Later, we will combine these values with some of the participants' performance data taken from the study dataframe to derive insights about what features of tasks determine how hard those tasks are to solve.

## 2 - Data Preparation and Cleaning

Real datasets are often messy! They can contain irrelevant data (which we can just ignore, as above); but they can also have errors and missing data. We need to make sure that we catch as many of these issues as possible before we analyse the data.

Spotting errors requires an understanding of what the data should be like. We can start by verifying that certain "facts" we know should be true about the data really do hold true in the actual dataset.

We know the following facts should be true, because of how the data was collected:

- There should be 6 unique participants
- Every participant should have 3 sessions of data
- Every session should have 24 unique trials per participant
- Every trial should start with a "trial:start" event, and end with an "trial:end" event
- Time taken for each trial should be no greater than 90s and no less than 0s.

We can check each one of these facts using pandas.

### There should be 6 unique participants

We can easily check if there are 6 unique participants using the `unique()` method on the dataframe.

I'll also use the `assert` keyword here. This python keyword goes before a boolean-valued expression and causes the script to fail if the expression evaluates to `False`. If the expression evaluates to `True`, the script will simply continue executing.

In [ ]:
assert len(study_df.participant_id.unique()) == 6

### Every participant should have 3 sessions of data

To check that each participant has 3 sessions, we can use the pandas `groupby` method. This method groups rows in the dataframe that have the same value in a given column, then computes some aggregated value for another column.

For example, here we group by "participant_id" to gather together all the rows for each participant.
Then we pick the "session_number" column for each group.
Then we apply the `unique()` method to the series of session numbers in each group.

This gives us a list of the unique session number values corresponding to each participant ID.

In [ ]:
unique_sessions_per_participant = study_df.groupby(["participant_id"]).session_number.unique()
unique_sessions_per_participant

We can see that all participants have the correct number of sessions (labelled 0, 1, and 2). If we had more than six participants, this would be hard to see by eye, so let's create some code to check it automatically.

One way is to use list comprehension to go through every row of the groupby result...

In [ ]:
[ss for ss in unique_sessions_per_participant]

...then we can check that each row has the right elements in it. We could do this by converting the elements to a set object (which is like a list where elements can't be repeated and their order doesn't matter), and then comparing that to the expected set {0, 1, 2}.

#### Side Note: Comparing Sets
Why convert to a set? Because we can't be sure that the elements in the lists generated above will be in the same order. We may have some like [1, 0, 2] and others like [0, 1, 2], and so on. If we try to compare these using '==' the result will be false, even though they have the same elements. Order matters in python lists, but not in sets.

In [ ]:
print([0, 1, 2] == [1, 0, 2])
print(set([0, 1, 2]) == set([1, 0, 2]))

If all of these comparisons return True, then we know that all participants have the right numbers of sessions.
Combining all of this, we have:

In [ ]:
expected_session_indices = set([0, 1, 2])
[set(ss) == expected_session_indices for ss in unique_sessions_per_participant]

In [ ]:
expected_session_indices = set([0, 1, 2])
assert all([set(ss) == expected_session_indices for ss in unique_sessions_per_participant])

### Every session should have 24 unique trials per participant

Pandas has another helpful aggregator that we can use here. It's called `nunique()`, and it counts the number of unique elements in each group of a groupby result.

In [ ]:
n_trials_per_participant_per_session = study_df.groupby(["participant_id", "session_number"]).trial_number.nunique()
n_trials_per_participant_per_session

Try to complete the following code to check if all the participants' sessions have the correct number of trials.

In this case, the result should be "False", because one participant has the wrong number of trials... (That's okay! Remember, we're trying to spot any issues in the data. We'll investigate why this participant has the wrong number of trials later.)

In [ ]:
n_trials_per_participant_per_session = study_df.groupby(["participant_id", "session_number"]).trial_number.nunique()
all([ ? for n in n_trials_per_participant_per_session])

#### Investigating the trial count issue

Before continuing to check the other properties of the dataset, we should determine why one participant has too many trials.

We see that only one participant has the wrong number of trials.

In [ ]:
# Count the number of unique trials per participant in each session
n_trials_per_participant_per_session = study_df.groupby(["participant_id", "session_number"]).trial_number.nunique()

# Find any participants and sessions for which the number of trials is not equal to 24
n_trials_per_participant_per_session[n_trials_per_participant_per_session != 24]

Looking at the participants unique trial numbers, we see that it contains the values 1-24 (as expected), but also contains the values -2 to 0.

In [ ]:
study_df[study_df.participant_id == "58d62c1733fef30001189b75"].trial_number.unique()

Checking the session that these extra trials occur in, we see that they were all in session 0.

In [ ]:
# Select the rows of the dataframe where the participant is the invalid participant and the trial number is in the range -2 to 0
invalid_trial_data = study_df[(study_df.participant_id == "58d62c1733fef30001189b75") & (study_df.trial_number.isin([-2, -1, 0]))]

# Find the unique session numbers in which invalid trial numbers occur.
invalid_trial_data.session_number.unique()

Let's look at the task signatures (i.e. the tuples containing the puzzle start and end states for each trial) and check if they are in our other dataframe containing all the non-practice task signatures.

In [ ]:
# Add a new column to the task features dataframe that contains task signature tuples.
# (At the moment we have start and target string as separate columns. Adding this new column makes comparisons with the trial data easier)
task_features_df["task_signature"] = task_features_df.apply(lambda r: (r.start_string, r.target_string), axis="columns")

# Check that the task signatures for the invalid rows are contained in the task features dataframe
for signature in invalid_trial_data.task_signature.unique():
    print(signature in task_features_df.task_signature.unique())

We see that none of these task signatures were in the dataframe containing the non-practice task signatures.

From our knowledge of how the data was collected, we know that there were three practice trials in session 0.

We can conclude, based on the available evidence, that these three extra trials were practice trials that were accidentally recorded alongside the non-practice trials.

To make sure we don't use the practice data in the analyses below, we will remove it from the dataframe.
Note that this won't change the CSV files, so we need to ensure any invalid data is removed whenever we load the data again.

In [ ]:
study_df = study_df[study_df.trial_number > 0]

Now try running your check for 24 trials per participant per session again. This time it should return "True".

### Every trial should start with a "trial:start" event, and end with an "trial:end" event

This one is a bit more challenging to check, but we can do it using a custom aggregator function.

Our custom function runs on a series of event_type values from the dataframe, and checks that they first is "trial:start" and the last is "trial:end".

The assertion passes, so all trials meet this criterion.

In [ ]:
def check_first_and_last_events(rows):
    first_is_start_event = rows.iloc[0] == "trial:start"
    last_is_end_event = rows.iloc[-1] == "trial:end"
    return first_is_start_event and last_is_end_event

assert all(study_df.groupby(["participant_id", "session_number", "trial_number"]).event_type.aggregate(check_first_and_last_events))

### Time taken for each trial should be no greater than 90s and no less than 0s

This one demonstrates a common issue with time data.

Timing events are often less precise than we would like. In this study, participants were not allowed to take longer than 90s to solve a trial. Ideally, we would be able to check that all trials took under 90s by comparing the timestamps of the "trial:end" events to 90s. But due to the limitations of timing precision, some trials may have lasted slightly longer than 90s, as we see below:

In [ ]:
study_df[study_df.event_type == "trial:end"].event_timestamp / 1000

This makes our assertion fail:

In [ ]:
assert all(study_df[study_df.event_type == "trial:end"].event_timestamp / 1000 < 90)

Checking how many trials have an end time over 90.5s, we see there is only one trial. It's also the very first trial in session 0 for this participant. 

It's not clear why this error happened, but it is unlikely to cause problems for our analysis. We can therefore keep this row in the dataframe and move on.

In [ ]:
end_events = study_df[study_df.event_type == "trial:end"]
end_events[end_events.event_timestamp / 1000 > 90.5]

Plotting a histogram of end times for trials that end after 90s (and ignoring the one extreme event above), we see that most of these trials had a recorded end time of less than 60ms after 90s. For our analyses, this level of error can be safely ignored.

In [ ]:
_ = plt.hist(end_events[(end_events.event_timestamp / 1000 > 90) & (end_events.event_timestamp / 1000 < 90.5)].event_timestamp / 1000, bins = 50)
plt.xlabel("End Time (s)")
plt.ylabel("Trial Count")

## 3 - Exploratory Analysis and Visualisation

Having cleaned the data, we can now do some initial exploratory analyses.

This is where we get a sense of whether there are any scientifically interesting things going on in the data.

Exploration is guided by the the questions we want to answer, but it doesn't have to be statistically robust. We will develop more robust analyses later. Exploration helps us choose which ones to spend time developing.

Our exploration is guided by a series of general questions about the data.

### How did solution time vary across participants and sessions?

If participants got better at the task (on average), we would expect per-participant median solution times to move closer to zero across sessions.

We can check this by plotting per-participant median solution times for successful trials.

We choose the median as it is more robust to outliers than the mean.

In [ ]:
# Compute the per-participant median solution times for successful trials
solved_trial_data = study_df[study_df.event_type == "goal:achieved"]
median_participant_session_solution_times = solved_trial_data.groupby(["participant_id", "session_number"]).event_timestamp.median()/1000

fig, ax = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(4, 2), constrained_layout=True)
ax.set_xticks([0, 1, 2])
ax.set_xlabel("Session")
ax.set_ylabel("Solution Time (s)")
ax.set_ylim([0, 90])

for pid in study_df.participant_id.unique():
    ax.plot(median_participant_session_solution_times.loc[pid, :], ".-")

It appears that, when participants solved trials correctly, the median time they took to do so changed relatively little (perhaps 10s) between sessions.
One participant shows a sizeable decrease from session 0 to 1, but this is not the general pattern.

It's plausible that solution time is close to floor (i.e. the minimum achievable value) for the participants by session 1.
It's also plausible that the kinds of tasks participants solved were similar in some way, so they all had similar solution times.

Data exploration often brings up new questions that we might not have thought to ask at first. For the sake of brevity, we'll stick to simple analyses in this notebook; but one could try to develop additional analyses to sense-check the two preceding hypotheses (before developing more robust analysis methods to verify whether any effects are statistically reliable).

### How did proportion of trials solved vary across participants and sessions?


It may be more informative to look at the proportion of trials solved to see if participants are getting better at solving the puzzles with practice.

Our visualisation method here is similar to above. This time we plot the proportion of trials that were solved.

In [ ]:
solved_trial_data = study_df[study_df.event_type == "goal:achieved"]
proportion_trials_solved = solved_trial_data.groupby(["participant_id", "session_number"]).event_timestamp.count() / 24

fig, ax = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(4, 2), constrained_layout=True)
ax.set_xticks([0, 1, 2])
ax.set_xlabel("Session")
ax.set_ylabel("Proportion Trials Solved")
ax.set_ylim([0, 1])

for pid in study_df.participant_id.unique():
    ax.plot(proportion_trials_solved.loc[pid, :], ".-")

Here we see that the proportion of trials being solved by participants appears to be increasing across sessions.

To verify that this effect is statistically robust (i.e. not likely to be due to random noise in the data) we could perform a hypothesis test to check for an increase between the first and final session. A better method would be to fit a (Bayesian logistic) regression model and evaluate the uncertainty in the slope parameter. If the slope is reliably positive, this suggests that the probability of solving a trial is indeed increasing across sessions. 

### Were some tasks harder than others?

Our central scientific question is about what properties of tasks determine their difficulty.

If there is no variation in the difficulty of the tasks in this dataset, we cannot answer this question.

Let's perform a rough check to see if some tasks were harder than others by computing the number of successes for each task and plotting a histogram of these numbers.

In [ ]:
# Select the rows corresponding to successful trials
selected_data = study_df[study_df.event_type == "goal:achieved"]

# Count the number of successes for each task
success_counts_df = selected_data.groupby(["task_signature"]).participant_id.count().reset_index()
success_counts_df.columns = ["task_signature", "n_participants_successful"]

# What does this do?
# Hint: What happens above when the goal is not achieved by any participant?
missing_tasks = set(study_df.task_signature.unique()).difference(success_counts_df.task_signature.unique())
missing_task_success_counts = pd.DataFrame({"task_signature":list(missing_tasks)})
missing_task_success_counts["n_participants_successful"] = 0

# Combine the two dataframes
all_success_counts = pd.concat([success_counts_df, missing_task_success_counts]).reset_index(drop=True)

Complete the code below to plot a histogram where the x-axis shows each possible number of participants (from 0 to 6) and the y-axis shows the number of tasks that were successfully completed by this many participants.

In [ ]:
_ = plt.hist(?, bins=range(8))
_ = plt.xlabel("N Participants")
_ = plt.ylabel("Number of Tasks Solved By N Participants")

The number of participants who completed each task (x-axis) is a rough approximation of the difficulty of the task: if more participants solved it, the task is likely to be easier.

You should see from the histogram that there are several tasks for each number of participants (i.e. each approximate level of difficulty). This suggests that the data may be suitable for answering out main question about what factors make the tasks easier or harder.

## 4 - Answering the Main Question

Having sense-checked our data, we're now ready to apply some more sophisticated analyses to try and answer the main scientific questions: what properties of tasks determine their difficulty?

We won't go into the technical details of the analysis method used here (you'll cover it in other modules, if you haven't already), and instead we'll use a popular analysis library to do most of the hard work for us.

We're going to use logistic regression to try and predict the probability (technically the log-odds) of successfully completing a puzzle as a linear function of the pre-computed features of that puzzle.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# Select some of the features from the dataframe to use as predictors of performance
chosen_features = ['start_state_length', 'start_state_symbols_count', 'state_graph_size', 
                   'min_solution_path_length', 'solution_path_count', 'min_children_of_solution_path_node', 
                   'min_children_of_solution_path_node_off_paths', 'proportion_dead_end_states']

# Create an indicator column to indicate whether the trial was successful or not
study_df["goal_achieved"] = study_df.groupby(['participant_id', 'session_number', 'trial_number'])['event_type'].transform(
    lambda x: any(x == 'goal:achieved')
)

In [ ]:
# Create a dataframe from which we can easily pick the feature values corresponding to each task signature
task_to_features = task_features_df.set_index("task_signature")
task_to_features = task_to_features[chosen_features]

In [ ]:
task_to_features

In [ ]:
# For each trial by each participant, get a copy of the features for the corresponding task.
# These will all be stacked into a 2D array of shape (trial, feature).
# We also add in another feature that counts the session in which the trial occurred.

trial_data = study_df[study_df.event_type == "trial:end"]
all_values = []
for _, row in trial_data.iterrows():
    values = task_to_features.loc[[(row.start_string, row.target_string)]].values[0]
    values = np.append(values, row.session_number)
    all_values.append(values)

In [ ]:
# Create the (trial, feature) array
X = np.stack(all_values)

# Create the array of outcomes (success = 1, failure = 0) for each trial
y = np.array(trial_data.goal_achieved.values, np.int16)

In [ ]:
# Split the data into training and testing sets.
# We'll use the test data to check how well the fitted model reproduces the observed successes or failures
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Transform each feature to have mean 0 and standard deviation of 1.
# This lets us compare their importance by comparing the fitted parameter values.
# It also helps avoid numerical issues with the optimisation algorithm behind the scenes.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the Logistic Regression model using the scaled training data!
model = LogisticRegression(penalty="l2")
model.fit(X_train_scaled, y_train)

# Use the fitted model to predict whether the test data trials were successful.
# Compute the accuracy score for the model based on this.
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
accuracy = accuracy_score(y_test, y_pred)

# Print the proportion of test set trials whose outcome was successfully predicted by the model.
print(f'Accuracy: {accuracy}')

For each of the selected features, we fit a parameter value indicating how much it influences the probability that a trial will be successful

In [ ]:
for cc in zip(list(model.coef_[0]), list(task_to_features.columns) + ["session"]):
    print(cc)

In this case (but not in all cases when using logistic regression) we can interpret the parameter values for each feature as indicators of how important that feature is to determining whether a trial is successful or not. This is how we determine which factors are most important for determining the difficulty of a task.

Positive parameters mean that, as the feature value increases, so does the probability of successfully completing that task.

Negative parameters mean that, as the feature value increases, the probability of successfully completing the task decreases.

In this case, the biggest positive influence on performance is the session number (which is a proxy for the amount of practice the participants have had); and the biggest negative influence is the minimum length of the solution path. So puzzles which require more steps to solve tend to be harder.

Have a look at the other parameter values. Do any of them seem surprising to you? If so, that's okay - unexpected results are interesting! And you can use the same tools of data science to try and figure out why they happened.

For our purposes, these results suggest some interesting hypotheses about the sources of difficulty in problem-solving tasks, which I will test in the future with new empirical studies.

## Closing Thoughts

Can we conclude from the preceding analysis that factors like number of solution paths and proportion of dead-end states causally influence task performance? Not quite.

Regression alone doesn't tell us about causality. Take a look at the code below to see why. This is the same model as above, but we haven't included the session number as a predictor. See how the fitted parameter values have changed? If we interpret this causally, we might now think that `start_state_symbols_count` has a positive influence on performance; yet, above, it had a negative influence, which doesn't make sense. Simply choosing to include different predictors shouldn't influence which ones are causally important.

If we want to make causal conclusions, we have two options: we can use more sophisticated analyses (e.g. structural causal modelling). Or we can do new experiments in which we try to control some factors and experimentally manipulate others (to see if the manipulations have an effect on the outcome when the other factors are kept constant). Which you choose depends on the goals of your project, the cost of collecting more data, and whether it is possible to achieve this level of experimental control. Often, modelling is the only viable option.

Developing effective models is an iterative process involving review, critique, and plenty of trial and error. As you learn more sophisticated analysis techniques, you will be able to design your own models to derive valuable (and potentially causal) insights from complex datasets. This requires intuition, technical understanding, and a lot of creativity. As you progress through your data science career, you'll develop all of these skills!

In [ ]:
chosen_features = ['start_state_length', 'start_state_symbols_count', 'state_graph_size', 
                   'min_solution_path_length', 'solution_path_count', 'min_children_of_solution_path_node', 
                   'min_children_of_solution_path_node_off_paths', 'proportion_dead_end_states']

study_df["goal_achieved"] = study_df.groupby(['participant_id', 'session_number', 'trial_number'])['event_type'].transform(
    lambda x: any(x == 'goal:achieved')
)

task_to_features = task_features_df.set_index("task_signature")
task_to_features = task_to_features[chosen_features]

trial_data = study_df[study_df.event_type == "trial:end"]
all_values = []
for _, row in trial_data.iterrows():
    values = task_to_features.loc[[(row.start_string, row.target_string)]].values[0]
    all_values.append(values)


X = np.stack(all_values)
y = np.array(trial_data.goal_achieved.values, np.int16)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LogisticRegression(penalty="l2")
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}\n')

for cc in zip(list(model.coef_[0]), list(task_to_features.columns)):
    print(cc)